Let's say we want to prepare data and try some scalers and classifiers for prediction in a classification problem. We will tune paramaters of classifiers by grid search technique.

Data preparing:

In [1]:
from sklearn.datasets import make_classification


X, y = make_classification()

Setting steps for our pipelines and parameters for grid search:

In [2]:
from reskit.core import Pipeliner


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


classifiers = [('LR', LogisticRegression()),
               ('SVC', SVC())]

scalers = [('standard', StandardScaler()),
           ('minmax', MinMaxScaler())]

steps = [('scaler', scalers),
         ('classifier', classifiers)]

param_grid = {'LR': {'penalty': ['l1', 'l2']},
              'SVC': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}}

Creating a plan of our research:

In [3]:
pipe = Pipeliner(steps, param_grid=param_grid)
pipe.plan_table

,scaler,classifier
0,standard,LR
1,standard,SVC
2,minmax,LR
3,minmax,SVC


To tune parameters of models and evaluate this models, run:

In [4]:
pipe.get_results(X, y, scoring=['roc_auc'])

Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4


,scaler,classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,standard,LR,0.954632,0.0368769,{'penalty': 'l1'},0.953607,0.0372264,[ 0.96885813 0.98961938 0.90234375]
1,standard,SVC,0.948824,0.0370676,{'kernel': 'rbf'},0.947989,0.0371741,[ 0.94117647 0.99653979 0.90625 ]
2,minmax,LR,0.959412,0.0378114,{'penalty': 'l1'},0.958369,0.0381575,[ 0.97231834 0.99653979 0.90625 ]
3,minmax,SVC,0.948971,0.0311831,{'kernel': 'rbf'},0.948286,0.031253,[ 0.94117647 0.98961938 0.9140625 ]
